# AtomGPT.org API (AGAPI) Client

https://github.com/atomgptlab/agapi

References to cite:

1. https://doi.org/10.1016/j.commatsci.2025.114063
2. https://doi.org/10.1007/s40192-025-00410-9

# Table of contents

1. Get your AtomGPT.org API key
2. General question-answer based on open source GPT-OSS-20b
3. Query JARVIS-DFT entries for a formula
4. JARVIS-DFT by element search
5. Get ALIGNN Predictions
6. Use ALIGNN-FF to relax atomic structure
7. Protein folding with ESM fold
8. XRD Pattern to atomic structure

Author: Kamal Choudhary (kchoudh2@jhu.edu)

Do you have any suggestions/requests for other tools? Raise a GitHub [issue](https://github.com/atomgptlab/agapi/issues/new)

If you like this project, don't forget to give a GitHub [star](https://github.com/atomgptlab/agapi/stargazers) and cite papers above

In [ ]:
# !pip install -q agapi jarvis-tools


## Get your AtomGPT API Key



Go to Website: https://atomgpt.org/

Navigate to Profile >> Settings >> Account >> API Keys >> Show/Create

It will look like sk-xxxxxxxxxxxxx paste below

In [ ]:
import yaml

# Load the YAML file
with open("credential.yaml", "r") as f:
    cred = yaml.safe_load(f)

api_key = cred["atomGPT-api-key"]


## General question-answer

In [ ]:
from agapi.client import Agapi
client = Agapi(api_key=api_key)
r = client.ask("Whats the capital of US")
print(r)

In [ ]:
import pandas as pd

## Query JARVIS-DFT entries for a formula, e.g. MoS2

In [ ]:
%%time
r = client.jarvis_dft_query(formula="MoS2")
print(pd.DataFrame(r['results']))


In [ ]:
print(r['results'][0].keys())

## JARVIS-DFT by element search

In [ ]:
r = client.jarvis_dft_query(search="-Mo-S") #Note - in front of elements
print(r)

# Get ALIGNN Predictions

In [ ]:
!wget https://raw.githubusercontent.com/atomgptlab/agapi/refs/heads/main/agapi/images/POSCAR -O POSCAR

In [ ]:
!ls

Note: We recommend avoiding systems with more than 20 atoms. AGAPI is still under active development, and support for larger systems will improve as we expand computational resources.

In [ ]:
%%time
r = client.alignn_query(file_path="POSCAR")

In [ ]:
r

## Use ALIGNN-FF to relax atomic structure

In [ ]:
%%time
r = client.alignn_ff_query(poscar_string=open("POSCAR").read())

In [ ]:

r

In [ ]:
from jarvis.io.vasp.inputs import Poscar
orig = Poscar.from_string(r['original'])
print(orig)

In [ ]:
relaxed = Poscar.from_string(r['relaxed'])
print(relaxed)

## Protein folding with ESM fold

In [ ]:
# Protein fold (returns binary content if format=zip)
zbytes = client.protein_fold_query(sequence="AAAAA", format="zip")
open("protein.zip", "wb").write(zbytes)

In [ ]:
!unzip protein.zip

In [ ]:
!ls

## XRD Pattern matching based atomic structure

`Lab6data.dat ` is a file for 2 `$\theta$` vs intensity for LaB6 with specific header `elements HAS ALL La,B`

In [ ]:
!wget https://raw.githubusercontent.com/atomgptlab/agapi/refs/heads/main/agapi/images/Lab6data.dat -O Lab6data.dat

The following is matching XRD patterns, DifractGPT model will be available soon

In [ ]:
r = client.pxrd_query(file_path="Lab6data.dat")
print(r)

## Generate STEM image for relaxed structure above for (001) plane (default)

In [ ]:
import matplotlib.pyplot as plt
import os, glob
from jarvis.analysis.stem.convolution_apprx import STEMConv
from jarvis.db.figshare import data, get_jid_data
import matplotlib.pyplot as plt
from jarvis.core.atoms import Atoms, ase_to_atoms, get_supercell_dims
from jarvis.core.lattice import get_2d_lattice
%matplotlib inline
#plt.switch_backend("agg")

#graphene
a = relaxed.atoms #Atoms.from_dict(get_jid_data("JVASP-667")["atoms"])
p = STEMConv(atoms=a,output_size=[150,150]).simulate_surface(a)

plt.imshow(p[0], interpolation="gaussian", cmap="plasma")
# plt.savefig("stem_example.png")
# plt.close()

## Generate X-ray diffraction pattern for structure above

In [ ]:
from jarvis.analysis.diffraction.xrd import XRD
theta,d_hkls,intens = XRD().simulate(atoms=relaxed.atoms)
import matplotlib.pyplot as plt
plt.bar(theta, intens)
plt.xlim(0, 90)
plt.xlabel(r'2$\theta$ (degrees)')
plt.ylabel('Intensity (a.u.)')
plt.show()
